<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/JaxCubicEOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import jax.numpy as jnp
import jax
import requests
import re
import string
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'
R=8.314

In [ ]:
water_props = requests.get('https://raw.githubusercontent.com/profteachkids/CHE2064/master/WaterProps.txt').text
extract_single_props = {'Molecular Weight' : 'Mw',
                 'Critical Temperature' : 'Tc',
                 'Critical Pressure' : 'Pc',
                 'Critical Volume' : 'Vc',
                 'Acentric factor' : 'w',
                 'Normal boiling point' : 'Tb',
                 'Heat of vaporization' : 'Hvap'}
extract_coeff_props={'Vapor Pressure' : 'Pvap'}

In [ ]:
single_props_pat = re.compile('^\s{,2}([\w\s]+?)\s+:\s+([-.0-9e+]+)\s+[\w\s/]*$', re.MULTILINE)
single_props = dict(single_props_pat.findall(water_props))
props={}
for k,v in extract_single_props.items():
  props[v]=float(single_props.pop(k))
print(props)

{'Mw': 18.015, 'Tc': 647.35, 'Pc': 22118230.0, 'Vc': 0.063494, 'w': 0.348, 'Tb': 373.15, 'Hvap': 40656800.0}


In [ ]:
coeffs_name_pat = re.compile("([\w ]+)\s[^\n]*?Equation.*?Coeffs:([- e\d.+]+)+?", re.DOTALL)
coeffs_name_strings = dict(coeffs_name_pat.findall(water_props))

coeffs_pat = re.compile('([-\de.+]+)')
for k,v in extract_coeff_props.items():
  coeffs = coeffs_pat.findall(coeffs_name_strings[k])
  for letter, value in zip(string.ascii_uppercase,coeffs):
    props[v+letter]=value

In [ ]:
two_pi=2*jnp.pi
one_third = 1/3
def cubic_roots(a, b, c):
    # Returns only the real roots of cubic equations with real coefficients
    # x**3 + a x**2 + b x + c = 0

    Q = (a * a - 3 * b) / 9
    R = (2 * a * a * a - 9 * a * b + 27 * c) / 54
    det = (R * R - Q ** 3)

    if (det < 0):
      theta = jnp.arccos(R / pow(Q, 1.5))
      x=jnp.array((jnp.cos(theta/3), jnp.cos((theta+two_pi)/3), jnp.cos((theta-two_pi)/3)))
      x = -2 * jnp.sqrt(Q)*x - a/3
      return x
    else:
        A = -jnp.sign(R) * (abs(R) + jnp.sqrt(det)) ** one_third
        B = 0 if A == 0 else Q / A
        return jnp.array([(A + B) - a / 3])

In [ ]:
two_pi=2*jnp.pi
one_third = 1/3
@jax.jit
def cubic_roots_jax(a, b, c):
    # Returns only the real roots of cubic equations with real coefficients
    # x**3 + a x**2 + b x + c = 0

    Q = (a * a - 3 * b) / 9
    R = (2 * a * a * a - 9 * a * b + 27 * c) / 54
    det = (R * R - Q ** 3)

    def roots3(v):
      theta = jnp.arccos(R / pow(Q, 1.5))
      x=jnp.array((jnp.cos(theta/3), jnp.cos((theta+two_pi)/3), jnp.cos((theta-two_pi)/3)))
      x = -2 * jnp.sqrt(Q)*x - a/3
      return x
    
    def roots1(v):
      A = -jnp.sign(R) * (abs(R) + jnp.sqrt(det)) ** one_third
      B = Q / A
      return jnp.array([(A + B) - a / 3, jnp.nan, jnp.nan])

    return jax.lax.cond(det < 0, roots3, roots1, (1))




In [ ]:
def SRK_P(V,T,props, eos='SRK'):
  Tr = T/props['Tc']
  w = props['w']
  alpha = {'SRK' : (1 + (0.48 + 1.574*w - 0.176*w**2)*(1-Tr**0.5))**2}
  sigma = {'SRK' : 1}
  epsilon = {'SRK' : 0}
  omega = {'SRK' : 0.08664}
  psi = {'SRK': 0.42748}
  a = psi[eos] * alpha[eos] * R**2 * props['Tc']**2 / props['Pc']
  b = omega[eos] * R * props['Tc'] / props['Pc']
  return 8.314*T/(V-b) - a/((V+epsilon[eos]*b)*(V+sigma[eos]*b))

In [ ]:
V=jnp.logspace(-4.8,-3, 100)

fig=make_subplots(rows=1,cols=1)
for T in range(275,801,25):
  P=SRK_P(V,T,props)
  fig.add_scatter(x=V, y=P, mode='lines', name=f'{T}')

fig.update_layout(xaxis_title='$molar\ volume\ (m^3/mol)$',
                  yaxis_title='$Pressure\ (Pa)$')

fig.update_layout(xaxis_type='log', yaxis_range=(-50e6,100e6))
fig.show()


In [ ]:
def Zv(P, T, props, eos='SRK'):
  Tr = T/props['Tc']
  Pr = P/props['Pc']
  w = props['w']
  alpha = {'SRK' : (1 + (0.48 + 1.574*w - 0.176*w**2)*(1-Tr**0.5))**2}
  sigma = {'SRK' : 1}
  epsilon = {'SRK' : 0}
  omega = {'SRK' : 0.08664}
  psi = {'SRK': 0.42748}
  a = psi[eos] * alpha[eos] * R**2 * props['Tc']**2 / props['Pc']
  b = omega[eos] * R * props['Tc'] / props['Pc']
  beta = b*P/(R*T)
  q = a/(b*R*T)

  return cubic_roots_jax(beta*(epsilon[eos]+sigma[eos])-1-beta,
                         sigma[eos]*epsilon[eos]*beta**2 - (1+beta)*beta*(epsilon[eos]+sigma[eos])+q*beta,
                         -(1+beta)*sigma[eos]*epsilon[eos]*beta**2 - q*beta**2)

In [ ]:
Zv(10e6, 600, props)*R*600/10e6

DeviceArray([4.2453892e-05, 3.7700948e-04, 7.9376681e-05], dtype=float32)

In [ ]:
0.09709445*R*298/101325

0.002374127714536393